Pseudo skeleton code for ACO algorithm applied to TSP (Mohsin,Thangavelu. (2021))

procedure (Find approximate solution for TSP)
    set parameters, initialize pheromone trails
    while termination condidtion not met do
        ConstructSolutions
            randomized constructive search
        ApplyLocalSearch %optional
        UpdateTrails
    end
end

Sections
Read tsp file 
Create LIST with the cities and coordinates
create ARRAY with distances between cities (the number of distances is a combination nCr 323 distancias)

Questions:
Whats the answer format? And array with the rute example city1, city4, city5....cityn

In [23]:
##import libraries
import math

following code shows script to read the content of the tsp file and store it in a list and each index is a tuple that contains city-n, x,y
[( ),( ),( ),( ),....................,( )]
index[n-1]   city_n    coordinatesX     coordinatesY
0               1       11003.611100    42102.500000           

In [24]:
# Open input file
cities = []
file = open('dj38.tsp', 'r')
lines = file.readlines()
for line in (lines[10:]):
    # Assumes the file format: <CityIndex> <X> <Y>
    city_index, x, y = map(float, line.strip().split())
    cities.append((city_index, x, y))
print(cities[1])


(2.0, 11108.6111, 42373.8889)


Following code shows the calculation of distances between the cities #intentar hacerlo como una matriz 

In [44]:
distances=[]
for i in range(0,38):
    for x in range (i+1,38):
        x1=cities[x][1] #identify x1 coordinate
        x2=cities[i][1] #identify x2 coordinate
        y1=cities[x][2] #identify y1 coordinate
        y2=cities[i][2] #identify y2 coordinate
        
        d=math.sqrt((x2-x1)**2+(y2-y1)**2) ##apply formula for the distance between two points
        d=((i,x),d) #first element (cities) is a tuple (i,x) and the second is a float (distance)
        distances.append(d)

Introduction to the solution 
%Ant system for solving the traveling salesman problem:
<Input of matrix D of distances>
<Initialization of the parameters of the algorithm: α, β, Q, p, and τ0>
m=n %Number of ants is equal to the number of towns
For i=1:n %For each edge
    For j=1:n
        If i~=j
            η(i,j)=1/D(i,j) %Visibility
            τ(i,j)= τ0 %Pheromone
        Else τ(i,j)=0
        End
    End
End


In [45]:
k = len(cities) #number of ants given the number of cities  
tau_t = .01  #number of virtual pheromone on the edge at t time, will experience changes
alpha = 0.1  #pheromone
beta = 1     #heuristic value
p = .5 #pheromone evaporation coefficient

38


BIO
MOHAMED Mohsin, AJANTH Thangavelu
On Embarrassingly Parallel MaxMin Ant Colony Optimization for
Traveling Salesperson Problem. 2021